In [1]:
import tensorflow
from util import PATH_FOR_OUR_TRAINING_DATA, UPPER_LIMIT_OF_IMAGES, TARGET_SHAPE
import pandas as pd
from DS_Generator import DS_Generator, smart_batches
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from util import PATH_FOR_OUR_TRAINING_DATA
from model_evaluation import mean_average_precision

In [2]:
df = pd.read_csv(PATH_FOR_OUR_TRAINING_DATA + "/data.csv", index_col=0)

In [3]:
train_ds, val_ds = DS_Generator().generate(df,0.1,1)

Since none Batch-size was specified we, took the 64 specified in utils.py
We threw away the datapoint with index 46861 


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=TARGET_SHAPE+(1,)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [ ]:
precision_before_training = mean_average_precision(model,train_ds,val_ds)
print(precision_before_training)

In [ ]:
model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

In [ ]:
mean_average_precision(model,val_ds)